In [ ]:
!python -m pip install -q pyserial

from MathTools import get_rotate_offset_position
from MathTools import RotateAxis
from MathTools import normalize_coordinates

from DDPGAgent import DDPGAgent

import numpy as np
import warnings
import serial, struct, time
warnings.filterwarnings('ignore')

## Classes

In [5]:
class Servo_lx16a:
    
    def __init__(self, ID, Port, Interlock = False):
        self.ID = ID # unique servo ID
        self.Port = Port # COM port features for later use 
        self.Interlock = Interlock # While Interlock = True, Servo will not move
        
    @staticmethod   
    def checksum(msg):
        #Checksum=~(ID+Length+Cmd+Prm1+...+PrmN)
        #If the numbers in the brackets exceeds 255,
        #Take the lowest one byte, "~" means Negation.
        s = sum(msg)
        s = ~s&255
        chksum = s.to_bytes(1, byteorder ='little')
        return chksum
    
    @staticmethod  
    def display_msg(msg):
        #dispalys bytes in python console not as ASCII symbols but as bytes
        #print(''.join(r'\x'+hex(letter)[2:] for letter in msg))
        pass
      
    @staticmethod     
    def time(current_position, target_position):
        #currently not used
        #range of time for Servo movement is 0~30000ms
        #3000ms for 1000 steps
        #y ms for N steps
        #y = N*3000/1000
        return abs(target_position-current_position)*3000/1000
    
    @staticmethod
    def to_angle(position):
        #returns Servo angle from Servo position
        #Range of position is 0..1000 which is equal to 0..240 deg
        #240 degree equals to 1000 
        #x degree equals to N
        #x= (N*240/1000)
        return position*240/1000
    
    @staticmethod
    def to_position(angle):
        #return position from angle
        return int(angle*1000/240)
    
    Header = b'\x55\x55' #every message transfered and recieved starts with this Header
    
    def ReadPosition(self):
        #Reads current position of Servo 
        temp = self.ID+b'\x03'+b'\x1c' 
        msg = self.Header + temp + self.checksum(temp) #message to read Servo position
        #print(f'Reading position of servo {int.from_bytes(self.ID, "little")}, sent:')
        self.display_msg(msg)

        wait = True
        data_in = b'' # receive answer in data_in while wait True
        ser.write(msg) #send message to COM
        while wait:
            while ser.in_waiting:
                data_in += ser.readline()
                #print('Received: ')
                self.display_msg(data_in)
                if len(data_in)>=8: wait = False
        received_chksum = self.checksum(data_in[-6:-1]) 
        if data_in.startswith(self.Header): #check that received data_in startswith Header and that checksum is correct
            #print('Message Header correct')
            if  (received_chksum == data_in[-1].to_bytes(1, 'big')):
                #print('Checksum correct')
                position = struct.unpack('<h', data_in[-3:-1])[0]
                angle = self.to_angle(position)
                #print(f'Current position {position}, {angle}')
                #print('---------------------------------------')
                return angle
            else:
                print('Bad checksum')
        else:
            print('Bad message header')



    def MoveServo(self, target_angle):
        #Moves servo to traget angle
        if self.Interlock:
            print('Servo is Interlocked') #if Servo is interlocked, it will not move
        else:
            target_position = self.to_position(target_angle) 
            temp = self.ID+b'\x07'+b'\x01'+struct.pack('<h', target_position)+struct.pack('<h', 1500) #set time temporary to 1500 ms for any movement
            msg = self.Header+temp+self.checksum(temp) # message to move servo to desired position
            #print(f'Move servo {int.from_bytes(self.ID, "little")} to {target_position}, sent:')
            self.display_msg(msg)
            self.Interlock = True #Interlock servo
            ser.write(msg) #sending message to COM
            #This while cycle switches Interlock to False, when Servo reaches target position
            while self.Interlock:
                current_angle = self.ReadPosition()
                angle_diff = abs(current_angle-target_angle)
                #print('Angle_difference:', angle_diff)
                #print('---------------------------------------')
                if angle_diff<0.8:
                    self.Interlock=False
                    #print('Reached target angle +- 1 deg')
                    #print('---------------------------------------')

In [6]:
test_result = get_rotate_offset_position(RotateAxis.OY, 0, RotateAxis.OX, 5, RotateAxis.OZ, 2,
                                             RotateAxis.OY, 0.4048796, [0, 0.8378, 0])
print(test_result)

[-0.0151087040722454, 0.8343491981978997, 0.07299609626426051]


## Servo settings

In [ ]:
ser.close()

In [8]:
Servo1_ID = b'\x01'
Servo2_ID = b'\x02'
Servo3_ID = b'\x03'

com = 'COM9' #COM port id|
baudrate = 115200 #COM speed

ser = serial.Serial(com, baudrate, timeout=0.2)
Servo1 = Servo_lx16a(Servo1_ID, ser)
Servo2 = Servo_lx16a(Servo2_ID, ser) # 0 - 110
Servo3 = Servo_lx16a(Servo3_ID, ser)

## Testing

In [9]:
state_size = 12
action_size = 3
agent = DDPGAgent(state_size = state_size - 3, action_size = action_size, goal_size = 3, 
                  action_high = 5, action_low = -5, 
                  actor_learning_rate = 1e-3, critic_learning_rate = 1e-3,
                  tau = 0.1)

In [10]:
class Servo():
    def __init__(self):
        self.angle = 0

    def rotate(self, delta_angle):
        self.angle += delta_angle


In [11]:
def read_state_test(servo1, servo2, servo3):
    # текущие углы роборуки
    firstAngle = servo1.angle;
    secondAngle = servo2.angle;
    thirdAngle = servo3.angle;
        
    # длины двух сочленений роборуки
    firstHandLength = 12 #0.4048796
    secondHandLength = 9 #0.4048796
    
    # общая длина руки в вытянутом состоянии
    allHandLength = firstHandLength + secondHandLength
    
    # позиция конца роборуки в вытянутом состоянии
    defaultEndPosition = [0, allHandLength, 0]
    
    # не надо нормализовать, тк мы находим позицию для настоящих углов и длин, после чего нормализуем найденную позицию
    #defaultEndPosition = normalize_coordinates(allHandLength, 1, defaultEndPosition)
    
    # поиск позиции конца роборуки по её углам и осям вращения
    endPosition = get_rotate_offset_position(RotateAxis.OY, firstAngle, 
                                             RotateAxis.OX, secondAngle, 
                                             RotateAxis.OX, thirdAngle,
                                             RotateAxis.OY, firstHandLength, 
                                             defaultEndPosition)
    
    # не надо нормализовать, из-за этого находится неправильная позиция
    #endPosition = normalize_coordinates(allHandLength, 1, endPosition)
    
    cubePosition = [0, 12, 0]
    #cubePosition = normalize_coordinates(allHandLength, 1, cubePosition)
    
    print(f'defaultEndPosition = {defaultEndPosition}; rotated endPosition = {endPosition}; cubePosition = {cubePosition}')
    
    # векторное расстояние между концом руки и целью
    vectorDistance = [cubePosition[0] - endPosition[0], 
                      cubePosition[1] - endPosition[1], 
                      cubePosition[2] - endPosition[2]]
    
    firstAngle = firstAngle/360
    secondAngle = secondAngle/360
    thirdAngle = thirdAngle/360
    
    state = [vectorDistance[0], vectorDistance[1], vectorDistance[2], 
              endPosition[0], endPosition[1], endPosition[2],
              firstAngle, secondAngle, thirdAngle]
    
    goal = [cubePosition[0], cubePosition[1], cubePosition[2]]
    return state, goal

In [12]:
def read_state():
    # так как рука в вытянутом состоянии имеет значения углов (120, 120, 120) (для возможности вращаться в обе стороны, тк имеет ограничение [0, 240])
    firstAngle = Servo1.ReadPosition() - 120 #Servo1.ReadPosition() + 120;
    secondAngle = Servo2.ReadPosition() - 120 #Servo2.ReadPosition() + 120;
    thirdAngle = Servo3.ReadPosition() - 120 #240 - Servo3.ReadPosition() + 120;
    
    print("\n firstAngle = ", firstAngle, " secondAngle = ", secondAngle, " thirdAngle = ", thirdAngle, "\n")
    
    # длины двух сочленений роборуки
    firstHandLength = 12
    secondHandLength = 9
    
    # общая длина руки в вытянутом состоянии
    allHandLength = firstHandLength + secondHandLength
    
    # позиция конца роборуки в вытянутом состоянии
    defaultEndPosition = [0, allHandLength, 0]
    
    # надо ли нормализовать?????
    #defaultEndPosition = normalize_coordinates(allHandLength, 1, defaultEndPosition)
    
    # поиск позиции конца роборуки по её углам и осям вращения
    endPosition = get_rotate_offset_position(RotateAxis.OY, firstAngle, 
                                             RotateAxis.OX, secondAngle, 
                                             RotateAxis.OX, thirdAngle,
                                             RotateAxis.OY, firstHandLength, 
                                             defaultEndPosition)
    
    print(f'not normalize end pos: {endPosition}')
    
    #not normalize end pos: [-0.0007017679341761611, 1.0011229079971835, 0.08376541700570701]
    #normalize end pos: [-3.341752067505529e-05, 0.04767251942843731, 0.0039888293812241436]
    
    #endPosition = normalize_coordinates(allHandLength, 1, endPosition)
    
    cubePosition = [-0.007918067592270591, 0.7898086317810685, 0.6300666310496583] #endPosition #[-10, 6, 0]
    
    #cubePosition = normalize_coordinates(allHandLength, 1, cubePosition)
    
    # векторное расстояние между концом руки и целью
    vectorDistance = [cubePosition[0] - endPosition[0], 
                      cubePosition[1] - endPosition[1], 
                      cubePosition[2] - endPosition[2]]
    
    firstAngle = firstAngle/360
    secondAngle = secondAngle/360
    thirdAngle = thirdAngle/360
    
    state = [vectorDistance[0], vectorDistance[1], vectorDistance[2], 
              endPosition[0], endPosition[1], endPosition[2],
              firstAngle, secondAngle, thirdAngle]
    
    goal = [cubePosition[0], cubePosition[1], cubePosition[2]]
    return state, goal

In [20]:


#Servo2.MoveServo(150) #end pos: [-0.007918067592270591, 0.7898086317810685, 0.6300666310496583] - повернулся в положительную сторону по z
#Servo2.MoveServo(80) #end pos: [0.002591194524373274, 0.7871754797239352, -0.6185985076056891] - повернулся в отрицательную сторону по z

#Servo3.MoveServo(150) #end pos: [0.04649725024516806, 2.4736152827448286, -5.550070567099335]
#Servo3.MoveServo(80) #end pos: [-0.05975396614459472, 3.583286569751135, 7.132437445610642]

#Servo3.MoveServo(40) #end pos: [-0.09056626981047014, 10.256364693800629, 10.81029922837861]
#Servo1.MoveServo(150) # против часовой, end pos: [5.444508565215012, 10.256364693800629, 9.339598392843824]
#Servo1.MoveServo(80) # по часовой, end pos: [-6.99032676138843, 10.210885907229931, 8.236879969108381]

state, goal = read_state()

done, distance = check_if_done(state[3:6], goal)



 firstAngle =  -40.31999999999999  secondAngle =  -0.23999999999999488  thirdAngle =  -80.4 

not normalize end pos: [-6.99032676138843, 10.210885907229931, 8.236879969108381]


In [14]:
def set_action(action):
    angle1 = Servo1.ReadPosition() + action[0]
    angle2 = Servo2.ReadPosition() + action[1]
    
    # так как 3 серва крутится в другую сторону, относительно 1 и 2 серв, то поворачиваем на -action[2]
    angle3 = Servo3.ReadPosition() - action[2]
    
    if (angle1 >=0 and angle1 <= 240):
        Servo1.MoveServo(angle1)
    if angle2 >=0 and angle1 <= 240:
        Servo2.MoveServo(angle2)
    if angle3 >=0 and angle1 <= 240:    
        Servo3.MoveServo(angle3)
    
def set_action_test(action, servo1, servo2, servo3):
    servo1.rotate(action[0])
    servo2.rotate(action[1])    
    servo3.rotate(action[2])
    
    return servo1, servo2, servo3

def check_if_done(hand, goal):
    hand = np.array(hand)
    goal = np.array(goal)
    dist = np.linalg.norm(hand - goal)
    if dist <= 0.05:
        return True, dist
    return False, dist

In [10]:
Servo1.MoveServo(120)
Servo2.MoveServo(120)
Servo3.MoveServo(120)

## Test actions

In [ ]:
n_episodes = 1

servo1 = Servo()
servo2 = Servo()
servo3 = Servo()

distances = []

test_rewards = []
agent.saver.restore(agent.sess, "model/without_her_rotate_scaled.ckpt")
for i_episode in range(n_episodes):
    
    state, goal = read_state_test(servo1, servo2, servo3)
    
    r = 0
    while True:
        action = agent.choose_action([state], [goal], 0)
        print("\n\nACTION = ", action, "\n\n")
        print("\n\nSTATE = ", state, "\n\n")
        
        # применяем углы к сервам
        servo1, servo2, servo3 = set_action_test(action, servo1, servo2, servo3)
        
        time.sleep(1)
        
        # получаем обновленный state
        next_state, next_goal = read_state_test(servo1, servo2, servo3)
        
        done, distance = check_if_done(next_state[3:6], goal)
        
        reward = -1
        
        if done:
            reward += 10
        
        r += reward
        
        print("\n old state = ", state, "\n action = ", action, "\n new state = ", next_state, "\n distance = ", distance,
              "\n current reward = ", reward, "\n cumulative reward = ", r, "\n")
        
        state = next_state
        if done:
            print("episode:", i_episode+1, "rewards: %.2f" % r, end="\r")
            test_rewards += [r]
            break
         
        distances.append(distance)
        plt.plot(distances, label="ddpg+her")
        plt.legend()
        plt.title("epoch success rate (over 20 episodes)")
        plt.show()

## Algorithm

In [11]:
Servo1.MoveServo(120)
Servo2.MoveServo(120)
Servo3.MoveServo(120)

In [ ]:
n_episodes = 1

test_rewards = []
agent.saver.restore(agent.sess, "model/without_her_rotate_scaled.ckpt")
for i_episode in range(n_episodes):
    state, goal = read_state()
    r = 0
    while True:
        action = agent.choose_action([state], [goal], 0)
        print("\n\nACTION = ", action)
        print("\n\nSTATE = ", state)
        # применяем углы к сервам
        #servo1, servo2, servo3 = set_action_test(action, servo1, servo2, servo3)
        
        set_action(action)
        
        # как-то ждем, пока сервы повернутся
        #time.sleep(5)
        
        # получаем обновленный state
        next_state, next_goal = read_state()
        
        done, distance = check_if_done(next_state[3:6], goal)
        
        reward = -1
        
        if done:
            reward += 10
        
        r += reward
        
        print("\n old state = ", state, "\n action = ", action, "\n new state = ", next_state, "\n distance = ", distance,
              "\n current reward = ", reward, "\n cumulative reward = ", r, "\n")
        
        state = next_state
        if done:
            print("episode:", i_episode+1, "rewards: %.2f" % r, end="\r")
            test_rewards += [r]
            break

INFO:tensorflow:Restoring parameters from model/without_her_rotate_scaled.ckpt


INFO:tensorflow:Restoring parameters from model/without_her_rotate_scaled.ckpt



 firstAngle =  -0.480000000000004  secondAngle =  -0.23999999999999488  thirdAngle =  -0.23999999999999488 

not normalize end pos: [-0.0003509073637783973, 1.0002807333103656, 0.04188550121170425]


ACTION =  [-4.96324921  5.          4.75140619]


STATE =  [-0.007567160228492194, -0.21047210152929707, 0.5881811298379541, -0.0003509073637783973, 1.0002807333103656, 0.04188550121170425, -0.0013333333333333443, -0.0006666666666666524, -0.0006666666666666524]

 firstAngle =  -5.760000000000005  secondAngle =  4.799999999999997  thirdAngle =  -4.560000000000002 

not normalize end pos: [-0.0961522998965016, 0.9580108136515386, 0.9532203461144535]

 old state =  [-0.007567160228492194, -0.21047210152929707, 0.5881811298379541, -0.0003509073637783973, 1.0002807333103656, 0.04188550121170425, -0.0013333333333333443, -0.0006666666666666524, -0.0006666666666666524] 
 action =  [-4.96324921  5.          4.75140619] 
 new state =  [0.08823423230423101, -0.1682021818704701, -0.3231537150647952, 